In [49]:
import pandas as pd
import requests
import json
from dateutil import parser

In [6]:
OANDA_URL = "https://api-fxpractice.oanda.com/v3"
API_KEY = "83408b92cdfee022ba7a5afac8696138-969e18ef786964fa39f81a43f94c0ac3"
ID = "101-004-30368907-001"

In [7]:
session = requests.Session()

In [8]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [11]:
params = dict(
    count = 10,
    granularity = "M5"
)

In [12]:
url = f"{OANDA_URL}/instruments/USD_JPY/candles"

In [17]:
response = session.get(url, params=params, data=None, headers=None)

In [18]:
response.json()

{'instrument': 'USD_JPY',
 'granularity': 'M5',
 'candles': [{'complete': True,
   'volume': 1390,
   'time': '2024-11-18T07:25:00.000000000Z',
   'mid': {'o': '154.743', 'h': '154.746', 'l': '154.696', 'c': '154.698'}},
  {'complete': True,
   'volume': 1688,
   'time': '2024-11-18T07:30:00.000000000Z',
   'mid': {'o': '154.698', 'h': '154.762', 'l': '154.684', 'c': '154.714'}},
  {'complete': True,
   'volume': 1660,
   'time': '2024-11-18T07:35:00.000000000Z',
   'mid': {'o': '154.712', 'h': '154.768', 'l': '154.690', 'c': '154.759'}},
  {'complete': True,
   'volume': 1203,
   'time': '2024-11-18T07:40:00.000000000Z',
   'mid': {'o': '154.756', 'h': '154.767', 'l': '154.694', 'c': '154.704'}},
  {'complete': True,
   'volume': 1605,
   'time': '2024-11-18T07:45:00.000000000Z',
   'mid': {'o': '154.704', 'h': '154.704', 'l': '154.581', 'c': '154.582'}},
  {'complete': True,
   'volume': 1934,
   'time': '2024-11-18T07:50:00.000000000Z',
   'mid': {'o': '154.582', 'h': '154.584', 'l'

In [22]:
def get_instruments():
    ins_url = f"{OANDA_URL}/accounts/{ID}/instruments"
    res = session.get(ins_url, data=None, headers=None)
    return res.json()

In [23]:
res = get_instruments()

In [28]:
instruments = res['instruments']

In [30]:
instruments[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [31]:
key_i = ["name", "type", "displayName", "pipLocation", "displayPrecision", "tradeUnitsPrecision", "marginRate"]

In [33]:
instruments_dict = {}
for i in instruments:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [36]:
with open ("../data/instruments.json", "w") as file: 
    file.write(json.dumps(instruments_dict, indent=2))

In [44]:
def get_candles(pair_name, count=10, granularity="M5"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count=count,
        granularity=granularity,
        price="MBA"
    )
    res = session.get(url, params=params, data=None, headers=None)
    data = res.json()

    if res.status_code == 200:
        if 'candles' not in data:
            data=[]
        else:
            data = data['candles']

    return res.status_code, data
    

In [50]:
code, data = get_candles(pair_name = "USD_JPY", count=500,granularity="M5")

In [51]:
data[0]

{'complete': True,
 'volume': 1454,
 'time': '2024-11-18T08:45:00.000000000Z',
 'bid': {'o': '154.841', 'h': '154.901', 'l': '154.819', 'c': '154.879'},
 'mid': {'o': '154.849', 'h': '154.910', 'l': '154.826', 'c': '154.888'},
 'ask': {'o': '154.857', 'h': '154.919', 'l': '154.833', 'c': '154.898'}}

In [66]:
def create_data_frame(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o','h','l','c']
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

In [68]:
def create_data_file(pair_name, count=10, granularity="M5"):
    code, data = get_candles(pair_name, count=count, granularity = granularity)

    if code != 200:
        print("Failed to fetch data: ", pair_name)
    elif len(data) == 0:
        print("No data: ", pair_name)

    candles_df = create_data_frame(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")

    print("Successful")
            

In [71]:
create_data_file("EUR_USD",count=500, granularity="H4")

Successful


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-11-18 08:45:00+00:00,1454,154.849,154.910,154.826,154.888,154.841,154.901,154.819,154.879,154.857,154.919,154.833,154.898
1,2024-11-18 08:50:00+00:00,1689,154.888,154.923,154.842,154.896,154.878,154.916,154.833,154.887,154.898,154.931,154.851,154.905
2,2024-11-18 08:55:00+00:00,1529,154.898,154.956,154.898,154.955,154.890,154.949,154.890,154.948,154.907,154.964,154.907,154.962
3,2024-11-18 09:00:00+00:00,1468,154.956,154.974,154.826,154.850,154.948,154.967,154.817,154.841,154.964,154.981,154.833,154.860
4,2024-11-18 09:05:00+00:00,1855,154.850,154.862,154.792,154.814,154.841,154.852,154.783,154.804,154.859,154.872,154.800,154.823


In [ ]:
def 